In [ ]:
"""
건축 관련 법령 Graph RAG - 다중 법령 지원 버전
건축법, 건축물관리법, 주택법, 국토계획법, 주차장법, 건축서비스산업진흥법
"""

import pdfplumber
import re
import os
from neo4j import GraphDatabase
from typing import List, Dict, Tuple
import logging
from dataclasses import dataclass

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# ==========================================
# 법령 정의
# ==========================================
@dataclass
class LawDefinition:
    """법령 정의"""
    code: str           # 짧은 코드 (예: BUILDING, HOUSING)
    name: str           # 법령명 (예: 건축법, 주택법)
    act_label: str      # Neo4j 레이블 (예: BuildingAct)
    decree_label: str
    rule_label: str
    pdf_paths: Dict[str, str]  # {'Act': 'path', 'Decree': 'path', 'Rule': 'path'}


# 지원하는 법령 목록
LAWS = {
    'BUILDING': LawDefinition(
        code='BUILDING',
        name='건축법',
        act_label='BuildingAct',
        decree_label='BuildingDecree',
        rule_label='BuildingRule',
        pdf_paths={
            'Act': '/home/jaehyeonpark/Downloads/건축법(법률)(제21065호)(20251001).pdf',
            'Decree': '/home/jaehyeonpark/Downloads/건축법 시행령(대통령령)(제35811호)(20251001).pdf',
            'Rule': '/home/jaehyeonpark/Downloads/건축법 시행규칙(국토교통부령)(제01531호)(20251031).pdf'
        }
    ),
    'BUILDING_MGMT': LawDefinition(
        code='BUILDING_MGMT',
        name='건축물관리법',
        act_label='BuildingMgmtAct',
        decree_label='BuildingMgmtDecree',
        rule_label='BuildingMgmtRule',
        pdf_paths={
            'Act': '/home/jaehyeonpark/Downloads/건축물관리법(법률)(제20549호)(20250604).pdf',
            'Decree': '/home/jaehyeonpark/Downloads/건축물관리법 시행령(대통령령)(제35549호)(20250604).pdf',
            'Rule': '/home/jaehyeonpark/Downloads/건축물관리법 시행규칙(국토교통부령)(제01495호)(20250602).pdf'
        }
    ),
    # 'HOUSING': LawDefinition(
    #     code='HOUSING',
    #     name='주택법',
    #     act_label='HousingAct',
    #     decree_label='HousingDecree',
    #     rule_label='HousingRule',
    #     pdf_paths={
    #         'Act': '/path/to/주택법.pdf',
    #         'Decree': '/path/to/주택법_시행령.pdf',
    #         'Rule': '/path/to/주택법_시행규칙.pdf'
    #     }
    # ),
    # 'LAND_PLAN': LawDefinition(
    #     code='LAND_PLAN',
    #     name='국토의계획및이용에관한법률',
    #     act_label='LandPlanAct',
    #     decree_label='LandPlanDecree',
    #     rule_label='LandPlanRule',
    #     pdf_paths={
    #         'Act': '/path/to/국토의계획및이용에관한법률.pdf',
    #         'Decree': '/path/to/국토의계획및이용에관한법률_시행령.pdf',
    #         'Rule': '/path/to/국토의계획및이용에관한법률_시행규칙.pdf'
    #     }
    # ),
    # 'PARKING': LawDefinition(
    #     code='PARKING',
    #     name='주차장법',
    #     act_label='ParkingAct',
    #     decree_label='ParkingDecree',
    #     rule_label='ParkingRule',
    #     pdf_paths={
    #         'Act': '/path/to/주차장법.pdf',
    #         'Decree': '/path/to/주차장법_시행령.pdf',
    #         'Rule': '/path/to/주차장법_시행규칙.pdf'
    #     }
    # ),
    # 'ARCH_SERVICE': LawDefinition(
    #     code='ARCH_SERVICE',
    #     name='건축서비스산업진흥법',
    #     act_label='ArchServiceAct',
    #     decree_label='ArchServiceDecree',
    #     rule_label='ArchServiceRule',
    #     pdf_paths={
    #         'Act': '/path/to/건축서비스산업진흥법.pdf',
    #         'Decree': '/path/to/건축서비스산업진흥법_시행령.pdf',
    #         'Rule': '/path/to/건축서비스산업진흥법_시행규칙.pdf'
    #     }
    # )
}


# ==========================================
# PDF 추출
# ==========================================
def extract_text_from_pdf(pdf_path: str, skip_toc=True) -> str:
    """PDF에서 텍스트 추출 (목차 제외)"""
    if not os.path.exists(pdf_path):
        logger.warning(f"파일 없음: {pdf_path}")
        return ""
    
    logger.info(f"📄 {os.path.basename(pdf_path)}")
    
    def is_toc_page(text: str) -> bool:
        """목차 페이지 판별"""
        if not text:
            return True
        lines = text.split('\n')
        article_lines = [l for l in lines if re.match(r'^\s*제\d+조', l)]
        has_clauses = bool(re.search(r'[①②③④⑤]', text))
        return len(article_lines) > 15 and not has_clauses
    
    text = ""
    skipped = 0
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if not page_text:
                    continue
                
                if skip_toc and is_toc_page(page_text):
                    skipped += 1
                    continue
                
                # 노이즈 제거
                lines = []
                for line in page_text.split('\n'):
                    line = line.strip()
                    if line in ["법제처", "국가법령정보센터"] or line.endswith("법"):
                        continue
                    if line.isdigit() or re.match(r'^법제처\s+\d+', line):
                        continue
                    lines.append(line)
                
                text += '\n'.join(lines) + "\n"
        
        logger.info(f"✅ {len(text):,}자 (목차 {skipped}p 제외)")
        return text
        
    except Exception as e:
        logger.error(f"PDF 추출 실패 {pdf_path}: {e}")
        return ""


# ==========================================
# 법령 파서
# ==========================================
class LawParser:
    """법령 파서"""
    
    def parse(self, text: str, law_code: str, law_type: str) -> Dict:
        """
        법령 파싱
        
        Args:
            text: PDF 텍스트
            law_code: 법령 코드 (예: BUILDING, HOUSING)
            law_type: 법령 종류 (Act, Decree, Rule)
        
        Returns:
            파싱된 데이터
        """
        articles_data = []
        
        pattern = re.compile(r'\n(제\d+조(?:의\d+)?)\(([^)]+)\)')
        matches = list(pattern.finditer(text))
        
        for i, match in enumerate(matches):
            article_id = match.group(1)
            title = match.group(2)
            
            start = match.start()
            end = matches[i+1].start() if i+1 < len(matches) else len(text)
            article_text = text[start:end].strip()
            
            clauses = self._parse_clauses(article_text)
            
            # 본문이 있는 조항만
            if clauses or len(article_text) > 100:
                articles_data.append({
                    'id': article_id,
                    'title': title,
                    'text': article_text,
                    'clauses': clauses
                })
        
        # 중복 제거
        unique = {}
        for art in articles_data:
            aid = art['id']
            if aid not in unique or len(art['text']) > len(unique[aid]['text']):
                unique[aid] = art
        
        result = list(unique.values())
        logger.info(f"✅ {law_code}-{law_type}: {len(result)}개 조항")
        
        return {
            'law_code': law_code,
            'law_type': law_type,
            'articles': result
        }
    
    def _parse_clauses(self, text: str) -> List[Dict]:
        """항 파싱"""
        clauses = []
        parts = re.split(r'([①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳])', text)
        
        i = 0
        while i < len(parts):
            part = parts[i]
            if re.match(r'[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', part):
                if i + 1 < len(parts):
                    clause_text = parts[i+1].strip()
                    if clause_text:
                        items = self._parse_items(clause_text)
                        clauses.append({
                            'id': part,
                            'text': clause_text,
                            'items': items
                        })
                i += 2
            else:
                i += 1
        
        return clauses
    
    def _parse_items(self, text: str) -> List[Dict]:
        """호 파싱"""
        items = []
        lines = text.split('\n')
        current_item = None
        current_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            match = re.match(r'^(\d+)\.\s', line)
            if match:
                if current_item and current_lines:
                    item_text = '\n'.join(current_lines)
                    subitems = self._parse_subitems(item_text)
                    items.append({
                        'id': current_item,
                        'text': item_text,
                        'subitems': subitems
                    })
                
                current_item = match.group(1)
                current_lines = [line]
            elif current_item:
                current_lines.append(line)
        
        if current_item and current_lines:
            item_text = '\n'.join(current_lines)
            subitems = self._parse_subitems(item_text)
            items.append({
                'id': current_item,
                'text': item_text,
                'subitems': subitems
            })
        
        return items
    
    def _parse_subitems(self, text: str) -> List[Dict]:
        """목 파싱"""
        subitems = []
        lines = text.split('\n')
        current_sub = None
        current_lines = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            match = re.match(r'^([가-힣])\.\s', line)
            if match:
                if current_sub and current_lines:
                    subitems.append({
                        'id': current_sub,
                        'text': '\n'.join(current_lines)
                    })
                
                current_sub = match.group(1)
                current_lines = [line]
            elif current_sub:
                current_lines.append(line)
        
        if current_sub and current_lines:
            subitems.append({
                'id': current_sub,
                'text': '\n'.join(current_lines)
            })
        
        return subitems


# ==========================================
# Neo4j Graph Builder
# ==========================================
class GraphBuilder:
    """Neo4j 그래프 구축"""
    
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        logger.info(f"✅ Neo4j 연결")
    
    def close(self):
        self.driver.close()
    
    def clear(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
        logger.info("⚠️  DB 초기화")
    
    def create_indexes(self):
        """인덱스 생성"""
        with self.driver.session() as session:
            indexes = [
                "CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.uid)",
                "CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.article_id)",
                "CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.law_code)",
                "CREATE INDEX IF NOT EXISTS FOR (c:Clause) ON (c.uid)",
                "CREATE INDEX IF NOT EXISTS FOR (i:Item) ON (i.uid)",
                "CREATE INDEX IF NOT EXISTS FOR (s:Subitem) ON (s.uid)",
            ]
            for idx in indexes:
                try:
                    session.run(idx)
                except:
                    pass
        logger.info("✅ 인덱스")
    
    def build(self, data: Dict, law_def: LawDefinition):
        """
        그래프 구축
        
        Args:
            data: 파싱된 데이터
            law_def: 법령 정의
        """
        law_code = data['law_code']
        law_type = data['law_type']
        
        # Neo4j 레이블 선택
        if law_type == 'Act':
            label = law_def.act_label
        elif law_type == 'Decree':
            label = law_def.decree_label
        else:
            label = law_def.rule_label
        
        logger.info(f"🔨 {law_code}-{law_type} ({label}) 구축...")
        
        with self.driver.session() as session:
            for art in data['articles']:
                self._build_article(session, art, law_code, law_type, label)
        
        logger.info(f"✅ {law_code}-{law_type}")
    
    def _build_article(self, session, art: Dict, law_code: str, law_type: str, label: str):
        """조항 및 하위 구조 생성"""
        art_uid = f"{law_code}_{law_type}_{art['id']}"
        
        # 조항 노드 (Article + 법령별 레이블)
        session.run(f"""
            MERGE (a:Article:{label} {{uid: $uid}})
            SET a.article_id = $id,
                a.title = $title,
                a.law_code = $law_code,
                a.law_type = $law_type,
                a.full_text = $text
        """, {
            'uid': art_uid,
            'id': art['id'],
            'title': art['title'],
            'law_code': law_code,
            'law_type': law_type,
            'text': art['text']
        })
        
        # 항
        for clause in art['clauses']:
            cls_uid = f"{art_uid}_{clause['id']}"
            
            session.run("""
                MERGE (c:Clause {uid: $uid})
                SET c.clause_id = $id,
                    c.content = $text,
                    c.law_code = $law_code,
                    c.law_type = $law_type
            """, {
                'uid': cls_uid,
                'id': clause['id'],
                'text': clause['text'],
                'law_code': law_code,
                'law_type': law_type
            })
            
            session.run("""
                MATCH (a:Article {uid: $a})
                MATCH (c:Clause {uid: $c})
                MERGE (a)-[:CONTAINS]->(c)
            """, {'a': art_uid, 'c': cls_uid})
            
            # 호
            for item in clause['items']:
                itm_uid = f"{cls_uid}_{item['id']}"
                
                session.run("""
                    MERGE (i:Item {uid: $uid})
                    SET i.item_id = $id,
                        i.content = $text,
                        i.law_code = $law_code,
                        i.law_type = $law_type
                """, {
                    'uid': itm_uid,
                    'id': item['id'],
                    'text': item['text'],
                    'law_code': law_code,
                    'law_type': law_type
                })
                
                session.run("""
                    MATCH (c:Clause {uid: $c})
                    MATCH (i:Item {uid: $i})
                    MERGE (c)-[:CONTAINS]->(i)
                """, {'c': cls_uid, 'i': itm_uid})
                
                # 목
                for sub in item['subitems']:
                    sub_uid = f"{itm_uid}_{sub['id']}"
                    
                    session.run("""
                        MERGE (s:Subitem {uid: $uid})
                        SET s.subitem_id = $id,
                            s.content = $text,
                            s.law_code = $law_code,
                            s.law_type = $law_type
                    """, {
                        'uid': sub_uid,
                        'id': sub['id'],
                        'text': sub['text'],
                        'law_code': law_code,
                        'law_type': law_type
                    })
                    
                    session.run("""
                        MATCH (i:Item {uid: $i})
                        MATCH (s:Subitem {uid: $s})
                        MERGE (i)-[:CONTAINS]->(s)
                    """, {'i': itm_uid, 's': sub_uid})
    
    def create_relations(self):
        """조항 간 관계 생성"""
        logger.info("🔗 관계 생성...")
        
        with self.driver.session() as session:
            # 1. 같은 법령 내 참조 (REFERS_TO)
            session.run("""
                MATCH (a:Article)
                WHERE a.full_text CONTAINS '제' AND a.full_text CONTAINS '조'
                WITH a, [x IN split(a.full_text, ' ') 
                         WHERE x =~ '제\\d+(?:의\\d+)?조.*'] AS refs
                UNWIND refs AS ref
                WITH a, head(split(ref, '조')) + '조' AS ref_id
                WHERE ref_id <> a.article_id AND ref_id =~ '제\\d+(?:의\\d+)?조'
                MATCH (t:Article {law_code: a.law_code, law_type: a.law_type, article_id: ref_id})
                MERGE (a)-[:REFERS_TO]->(t)
            """)
            
            # 2. 위임 관계 (DELEGATES_TO) - 각 법령별
            for law_code in LAWS.keys():
                # 령 -> 법
                session.run(f"""
                    MATCH (d:Article {{law_code: '{law_code}', law_type: 'Decree'}})
                    WHERE d.full_text CONTAINS '법 제'
                    WITH d, [x IN split(d.full_text, ' ') WHERE x =~ '법제\\d+조.*'] AS refs
                    UNWIND refs AS ref
                    WITH d, '제' + substring(ref, 2) AS ref_str
                    WITH d, head(split(ref_str, '조')) + '조' AS ref_id
                    WHERE ref_id =~ '제\\d+(?:의\\d+)?조'
                    MATCH (a:Article {{law_code: '{law_code}', law_type: 'Act', article_id: ref_id}})
                    MERGE (d)-[:DELEGATES_TO]->(a)
                """)
                
                # 규칙 -> 령
                session.run(f"""
                    MATCH (r:Article {{law_code: '{law_code}', law_type: 'Rule'}})
                    WHERE r.full_text CONTAINS '영 제'
                    WITH r, [x IN split(r.full_text, ' ') WHERE x =~ '영제\\d+조.*'] AS refs
                    UNWIND refs AS ref
                    WITH r, '제' + substring(ref, 2) AS ref_str
                    WITH r, head(split(ref_str, '조')) + '조' AS ref_id
                    WHERE ref_id =~ '제\\d+(?:의\\d+)?조'
                    MATCH (d:Article {{law_code: '{law_code}', law_type: 'Decree', article_id: ref_id}})
                    MERGE (r)-[:DELEGATES_TO]->(d)
                """)
            
            # 3. 법령 간 참조 (CROSS_REFERS_TO)
            # 예: 건축법 -> 주택법, 건축법 -> 국토계획법 등
            cross_patterns = [
                ('BUILDING', '주택법', 'HOUSING'),
                ('BUILDING', '건축물관리법', 'BUILDING_MGMT'),
                ('BUILDING', '국토의계획및이용에관한법률', 'LAND_PLAN'),
                ('BUILDING', '주차장법', 'PARKING'),
                ('HOUSING', '건축법', 'BUILDING'),
                ('LAND_PLAN', '건축법', 'BUILDING'),
            ]
            
            for from_code, keyword, to_code in cross_patterns:
                session.run("""
                    MATCH (a:Article {law_code: $from_code})
                    WHERE a.full_text CONTAINS $keyword
                    MATCH (t:Article {law_code: $to_code})
                    WITH a, t, 
                         [x IN split(a.full_text, ' ') 
                          WHERE x =~ '제\\d+(?:의\\d+)?조.*'] AS refs
                    UNWIND refs AS ref
                    WITH a, t, head(split(ref, '조')) + '조' AS ref_id
                    WHERE t.article_id = ref_id
                    MERGE (a)-[:CROSS_REFERS_TO]->(t)
                """, {'from_code': from_code, 'keyword': keyword, 'to_code': to_code})
        
        logger.info("✅ 관계")
    
    def stats(self):
        """통계 출력"""
        with self.driver.session() as session:
            # 법령별 통계
            law_stats = session.run("""
                MATCH (a:Article)
                RETURN a.law_code as law, a.law_type as type, count(a) as cnt
                ORDER BY law, type
            """).data()
            
            total_articles = session.run("MATCH (a:Article) RETURN count(a)").single()[0]
            total_clauses = session.run("MATCH (c:Clause) RETURN count(c)").single()[0]
            total_items = session.run("MATCH (i:Item) RETURN count(i)").single()[0]
            total_subitems = session.run("MATCH (s:Subitem) RETURN count(s)").single()[0]
            
            contains = session.run("MATCH ()-[r:CONTAINS]->() RETURN count(r)").single()[0]
            refers = session.run("MATCH ()-[r:REFERS_TO]->() RETURN count(r)").single()[0]
            delegates = session.run("MATCH ()-[r:DELEGATES_TO]->() RETURN count(r)").single()[0]
            cross_refers = session.run("MATCH ()-[r:CROSS_REFERS_TO]->() RETURN count(r)").single()[0]
            
            print("\n" + "="*70)
            print("📊 Graph Database 통계")
            print("="*70)
            
            print("\n법령별 조항 수:")
            current_law = None
            for stat in law_stats:
                if stat['law'] != current_law:
                    if current_law:
                        print()
                    current_law = stat['law']
                    print(f"\n{LAWS[stat['law']].name}")
                print(f"  {stat['type']:8s}: {stat['cnt']:4d}개")
            
            print(f"\n\n전체 노드:")
            print(f"  Article  : {total_articles:,}")
            print(f"  Clause   : {total_clauses:,}")
            print(f"  Item     : {total_items:,}")
            print(f"  Subitem  : {total_subitems:,}")
            print(f"  합계     : {total_articles+total_clauses+total_items+total_subitems:,}")
            
            print(f"\n관계:")
            print(f"  CONTAINS       : {contains:,}")
            print(f"  REFERS_TO      : {refers:,}")
            print(f"  DELEGATES_TO   : {delegates:,}")
            print(f"  CROSS_REFERS_TO: {cross_refers:,}")
            print(f"  합계           : {contains+refers+delegates+cross_refers:,}")
            print("="*70 + "\n")


# ==========================================
# 메인 실행
# ==========================================
def main():
    """메인 실행"""
    
    # Neo4j 설정
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "DxI3O9BGnjGjdgu800HRd8kewNhHU9URb6lCMn3V4XI"
    
    # 처리할 법령 선택, 필요시 수정
    # 우선은 2개 법령만 처리
    laws_to_process = ['BUILDING', 'BUILDING_MGMT']

    
    parser = LawParser()
    builder = GraphBuilder(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    try:
        # builder.clear()  # 기존 데이터 삭제 (선택)
        builder.create_indexes()
        
        # 각 법령 처리
        for law_code in laws_to_process:
            law_def = LAWS[law_code]
            
            logger.info(f"\n{'='*70}")
            logger.info(f"{law_def.name} 처리")
            logger.info(f"{'='*70}")
            
            # 법/령/규칙 각각 처리
            for law_type, pdf_path in law_def.pdf_paths.items():
                if not os.path.exists(pdf_path):
                    logger.warning(f"⚠️  파일 없음: {pdf_path}")
                    continue
                
                text = extract_text_from_pdf(pdf_path, skip_toc=True)
                if text:
                    parsed = parser.parse(text, law_code, law_type)
                    builder.build(parsed, law_def)
        
        # 관계 생성
        logger.info(f"\n{'='*70}")
        builder.create_relations()
        
        # 통계
        builder.stats()
        
        logger.info("✅ 모든 작업 완료!")
        
    except Exception as e:
        logger.error(f"❌ {e}", exc_info=True)
    finally:
        builder.close()


if __name__ == "__main__":
    main()

INFO - ✅ Neo4j 연결
INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX IF NOT EXISTS FOR (e:Article) ON (e.uid)` has no effect.} {description: `RANGE INDEX index_9755e5ee FOR (e:Article) ON (e.uid)` already exists.} {position: None} for query: 'CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.uid)'
INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE RANGE INDEX IF NOT EXISTS FOR (e:Article) ON (e.article_id)` has no effect.} {description: `RANGE INDEX index_8d08c679 FOR (e:Article) ON (e.article_id)` already exists.} {position: None} for query: 'CREATE INDEX IF NOT EXISTS FOR (a:Article) ON (a.article_id)'
INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlre


📊 Graph Database 통계

법령별 조항 수:

건축법
  Act     :  101개
  Decree  :   73개
  Rule    :   30개


건축물관리법
  Act     :   52개
  Decree  :   38개
  Rule    :   19개


전체 노드:
  Article  : 313
  Clause   : 1,045
  Item     : 1,594
  Subitem  : 295
  합계     : 3,247

관계:
  CONTAINS       : 2,934
  REFERS_TO      : 1,076
  DELEGATES_TO   : 0
  CROSS_REFERS_TO: 49
  합계           : 4,059

